In [31]:
import importlib
from functools import partial
from random import random
import networkx as nx
import pandas as pd
from pathlib import Path
from dataclasses import fields

In [32]:
import simulation as sm
importlib.reload(sm)

<module 'simulation' from '/home/sandeep/abm-trust/new/simulation.py'>

In [33]:
graph = nx.barabasi_albert_graph(1000, 8)

In [34]:
sm.graph_name = 'test.graph'
model = sm.setup_model()

KeyError: 'graph_path'

In [ ]:
for i in range(500):
    model.step()
    model.collect_data()

In [ ]:
model_df = model.datacollector.get_model_vars_dataframe()

In [65]:
from itertools import product
from pathlib import Path

# write config file
def write_params_file(params, fname):
    with open(fname, 'w', encoding='utf-8') as f:
        for k,v in params:
            f.write(f'--{k}={v}')

costs = [x/10 for x in range(0, 10)]
num_groups = [10]
#num_groups = [10, 20, 50]
num_tags = [2]
#num_tags = [2, 3, 5]
migration_wait_periods = [0]
#migration_wait_periods = [0, 2]
trust_thresholds = [0.1, 0.3, 0.5, 0.7, 0.9]

params = {
    "num_agents": 1000,
    "game_type": "NISD",
    "mutation_prob": 0.01,
    "timesteps": 2000,
    "runs": 50,
    "data_to_save": 0.25,
    "max_group_ratio": 0.25,
    "decay": 0.05,
    "cooperation_threshold": 0.5,
    "output_dir": "results",
    "graph_path": 'networks/sn_d8.graph',
    "migrate": True,
    'cost': 0.1,
    'num_groups': 10,
    'num_tags': 2,
    'trust_threshold': 0.5,
    'migration_wait_period': 0,
}

param_combinations = list(product(costs, num_groups, num_tags, migration_wait_periods, trust_thresholds))
for c, ng, nt, mwp, th in param_combinations:
    run_id = hash((c, ng, nt, mwp, th)) % (2 ** 31 - 1)
    params['run_id'] = run_id
    
    params.update({'cost': c, 'num_groups': ng, 'num_tags': nt, 'migration_wait_period': mwp, 'trust_threshold': th})
    
    params_fname =f"{params['output_dir']}/{run_id}.params"
    path = Path(params_fname)
    assert not path.exists(), "file already exists"
    
    with open(params_fname, 'w', encoding='utf-8') as f:
        f.writelines((f'--{k}={v}\n' for k,v in params.items()))

In [66]:
from games import GameType

In [68]:
game_type = GameType("nisd")

In [69]:
game_type

<GameType.NISD: 'nisd'>

In [71]:
GameType("NISD".lower())

<GameType.NISD: 'nisd'>

In [87]:
df = pd.read_parquet('../results/group_trust_migration/2084091263_3_group.parquet')

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 500 entries, (0, 151) to (9, 200)
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   size             500 non-null    float64
 1   cooperators      500 non-null    float64
 2   trustworthiness  500 non-null    float64
dtypes: float64(3)
memory usage: 15.6 KB


In [88]:
df

size  cooperators  trustworthiness
group timestep                                     
0     151        95.0    62.745098         0.517363
      152       102.0    58.947368         0.449758
      153        97.0    50.000000         0.504006
      154        90.0    51.546392         0.509978
      155        73.0    64.444444         0.589870
...               ...          ...              ...
9     196       103.0    60.185185         0.470846
      197        93.0    59.223301         0.501160
      198        88.0    51.612903         0.483898
      199        91.0    38.636364         0.411685
      200        98.0    54.945055         0.406334

[500 rows x 3 columns]

In [89]:
import numpy as np

In [90]:
np.random.seed(480321_2)

In [91]:
np.random.rand()

0.1677596061483343